In [1]:
pip install jmespath parsel unidecode

     ------------------------------------ 235.5/235.5 kB 801.6 kB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import requests
import jmespath
from parsel import Selector

In [13]:
def SaveJSON(fileName,JsonComic):
  print(fileName+"\n")
  with open(f"{fileName}.json", "w") as outfile:
      json.dump(JsonComic, outfile,indent= 2)


def GetComicJSON(urlComic):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}')
  print(response.status_code)
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()

  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Title": props.pageProps.comic[0].nameComic,
        "URL": props.pageProps.comic[0].urlComic,
        "CoverImage":   props.pageProps.comic[0].imgComic,
        "CreateAt":  props.pageProps.comic[0].create_date
        "statusComic": props.pageProps.comic[0].statusComic
        "Category":   props.pageProps.comic[0].category[*].name_cate,
        "LastChapter": props.pageProps.comic[0].lastChapter
        "TotalChapter": props.pageProps.comic[0].totalChapter
        "Chapters": props.pageProps.listChapter[0].{urlChapter:urlChapter, dateUpdateChapter: dateUpdateChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data

ListComic = {"comic":[]}
def fetchData(start, end):
    global ListComic
    starting = True
    idx = 1
    expression = jmespath.compile(
        """
        {
        "comic": props.pageProps.dataComicNew[*].
        {
            urlComic:urlComic,
            totalChapter:totalChapter,
            createDateComic:createDateComic,
            lastUpdateChapter:lastUpdateChapter
            }
        }
        """
    )
    idx = start
    while idx<=end:
        print(f"fetching {idx}\n")
        # start = time.time()
        response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/tim-truyen?status=2&sort=2&page={idx}')
        idx+=1
        print("status",response.status_code)
        selector = Selector(text=response.text)
        json_data = selector.css("script[type='application/json']::text").get()
        data = json.loads(json_data)
        flat_data = expression.search(data)
        if(flat_data["comic"] == []): break
        ListComic["comic"].extend(flat_data["comic"])
        # end = time.time()
        # eplap = end - start

        for oneComic in flat_data["comic"]:
            urlComic = oneComic["urlComic"]
            JsonComic = GetComicJSON(urlComic)
            SaveJSON(f"ComicData/{urlComic}",JsonComic)
    
    return ListComic

In [14]:
fetchData(1, 5)

fetching 1

status 200
200
ComicData/toi-se-bi-tru-khu-cung-hoang-de

200
ComicData/hay-de-toi-duoc-hieu-em

200
ComicData/toi-tuong-ban-than-khong-con-song-duoc-bao-lau

200
ComicData/gia-dinh-chong-bi-am-anh-boi-toi

200
ComicData/tinh-yeu-cua-ik-seob

200
ComicData/phuong-phap-che-giau-dua-con-cua-hoang-de

200
ComicData/danh-cho-nhan-vat-bi-bo-roi-yeu-thich-nhat-cua-toi

200
ComicData/serena

200
ComicData/vo-luyen-dinh-phong

200
ComicData/quang-doi-con-lai-cua-nu-phu-yeu-menh

200
ComicData/thieu-soai-vo-ngai-lai-bo-tron

200
ComicData/tu-do-trong-mo

200
ComicData/em-chi-muon-hit-van-khi-cua-anh

200
ComicData/kiep-nay-toi-nhat-dinh-tro-thanh-gia-chu

200
ComicData/lan-nua-toa-sang

200
ComicData/the-thao-cuc-han

200
ComicData/tro-thanh-co-chau-gai-bi-khinh-miet-cua-gia-toc-vo-lam

200
ComicData/su-phan-boi

200
ComicData/cha-oi-con-khong-muon-ket-hon-dau

200
ComicData/cuoc-hon-nhan-nay-du-sao-cung-se-tan-vo-ma-thoi

200
ComicData/dai-quan-gia-la-ma-hoang

200
ComicData/quyen-

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
for oneComic in flat_data["comic"]:
  print(oneComic["urlComic"])

In [ ]:
GetChapterOfComicJSON("cuop-dau", chapter=1)


In [9]:
def GetChapterOfComicJSON(urlComic, chapter=1):
  response = requests.get(f'https://doctruyenonline.vn/truyen-tranh/{urlComic}/chapter-{chapter}')
  print(response.status_code)
  
  selector = Selector(text=response.text)
  json_data = selector.css("script[type='application/json']::text").get()
  data = json.loads(json_data)
  expression = jmespath.compile(
      """
      {
        "Page": props.pageProps.dataContentChapter[*].{indexImg:indexImg,imgChapter:imgChapter}
      }
      """
  )
  flat_data = expression.search(data)
  return flat_data

In [10]:
JsonComic = GetComicJSON("toi-se-bi-tru-khu-cung-hoang-de")

In [ ]:
JsonComic

In [ ]:
# encodedUnicode = json.dumps(JsonComic, ensure_ascii=False)


In [ ]:
import unidecode
import re
def CreateSlug(accented_string:str):

  # accented_string is of type 'unicode'
  unaccented_string = unidecode.unidecode(accented_string).lower()
  unaccented_string = re.sub(r"[^\w\s]", '', unaccented_string)
  unaccented_string=re.sub(r"\s+","-",unaccented_string)
  return unaccented_string

CreateSlug("Tu Ti\u00ean \u1ede Th\u1ebf Gi\u1edbi Si\u00eau N\u0103ng L\u1ef1c")

In [ ]:
from bs4 import BeautifulSoup

# Assuming we have an HTML document in the 'html_content' variable
soup = BeautifulSoup(html_content, 'html.parser')

# Find the desired element and extract its text using get_text()
elements = soup.find_all('span',itemprop="name")

#get_text with strip set to true
for idx,element in  enumerate(elements):
  print(f"({idx},{element.text})")